In [1]:
import datasets
import pandas as pd

dataset = datasets.load_dataset("trl-lib/tldr")

dataset['train'][0]

/workspace/hello-rl/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'prompt': "SUBREDDIT: r/relationships\n\nTITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting\n\nPOST: Not sure if this belongs here but it's worth a try. \n\nBackstory:\nWhen I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. \n\nNow: Its been almost 3 years now and I've gotten better after counselling and mild anti depressants. My m

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [15]:
# Calculate token lengths for the dataset
print("Calculating token lengths...")

def to_chat_template(prompt, completion):
    return tokenizer.apply_chat_template([
        {"role": "system", "content": prompt},
        {"role": "assistant", "content": completion},
    ], tokenize=False)

def get_token_lengths(rows):
    prompts = rows["prompt"]
    completions = rows["completion"]
    
    # Tokenize prompts and completions
    total_lengths = [tokenizer(to_chat_template(prompt, completion), return_length=True)["length"] for prompt, completion in zip(prompts, completions)]
    
    return {"total_lengths": total_lengths}

# Map function to get token lengths
token_lengths = dataset.map(get_token_lengths, batched=True, batch_size=100)

# Calculate statistics
total_lengths = token_lengths["train"]["total_lengths"]

# Calculate max and 95th percentile
max_total_length = max(total_lengths)
p95_total_length = int(torch.tensor(total_lengths, dtype=torch.float32).quantile(0.95).item())

print(f"Max token lengths: total={max_total_length}")
print(f"P95 token lengths: total={p95_total_length}")


Calculating token lengths...
Max token lengths: total=[618]
P95 token lengths: total=523


In [30]:
print("Converting training dataset to tokens...")
def tokenize_chat(rows):
    chats = [to_chat_template(prompt, completion) for prompt, completion in zip(rows["prompt"], rows["completion"])]

    return tokenizer(
        chats,
        padding="max_length",
        truncation=True,
        # 618 is the max length of the dataset
        max_length=618,
        return_tensors="pt"
    )

tokenized_dataset = dataset.map(tokenize_chat, batched=True)

Converting training dataset to tokens...


In [31]:
# Drop the original prompt and completion columns as they're no longer needed
# after tokenization to save memory
columns_to_remove = ["prompt", "completion", "attention_mask"]
tokenized_dataset = tokenized_dataset.remove_columns(columns_to_remove)

print(f"Columns after removal: {tokenized_dataset['train'].column_names}")


Columns after removal: ['input_ids']


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModelling
import wandb
import os
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

output_dir = f"Qwen2-0.5B-SFT-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"

wandb.login(key=os.environ["WANDB_API_KEY"])

data_collator = DataCollatorForLanguageModelling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    report_to = "wandb", 
    save_steps = 500,
    save_total_limit = 5,
    num_train_epochs=3,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
)

trainer.train()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids.